In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as fm
import matplotlib
import pickle
import timeit
import h5py
import math
import os
import json
# import seaborn as sns
%matplotlib inline
import datetime
from os import listdir
from os.path import isfile, join

In [2]:
def average(day, hour, stop):
    return ans[day][hour][stop]

def hour(x):
    x = x/1000000
    date = datetime.datetime.fromtimestamp(x).strftime("%H")
    return(int(date))

def stops(file):
    maxi = 0
    for name, group in file.groupby(['VehicleJourneyID','TimeFrame','Instance']):
        if group['StopID'].nunique() == file['StopID'].nunique():
            return group['StopID'].unique()
        maxi = max(group['StopID'].nunique(), maxi)
        if group['StopID'].nunique() == maxi:
            group.sort_values(['Timestamp'], inplace = True)
            all_stops = group['StopID'].unique()
            if group['StopID'].nunique() == file['StopID'].nunique():
                break
    return all_stops

def calc(group):
    group.sort_values(['Timestamp'], inplace = True)  
    group = group[['VehicleJourneyID','TimeFrame','StopID','Timestamp','AtStop', 'Instance', 'Direction']]
    return group

def compu(df):
    total = []
    dict_real = {}
    unique_stops = stops(df)
    for name, group in df.groupby(['VehicleJourneyID','TimeFrame', 'Instance']):
        group = calc(group)
        dict_real[name] = {}
        for i in range(len(unique_stops)):
            if i != len(unique_stops)-1:
                start = group['Timestamp'].loc[group['StopID'] == unique_stops[i]].min()
                stop = group['Timestamp'].loc[group['StopID'] == unique_stops[i+1]].min()
                dict_real[name][unique_stops[i]] = stop-start
            else:
                dict_real[name][unique_stops[i]] = 0
        total += [group]
    complete = pd.concat(total)
    return complete, dict_real, unique_stops

def func(group):
    return group.loc[group['Timestamp'] == group['Timestamp'].min()]

def weekday(x):
    x = x/1000000
    x = int(x)
    if datetime.datetime.fromtimestamp(x).strftime("%A") == "Monday":
        return 1
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Tuesday":
        return 2
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Wednesday":
        return 3
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Thursday":
        return 4
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Friday":
        return 5
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Saturday":
        return 6
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Sunday":
        return 7
    
def make_dict(complete):
    dicty = {}
    for i in complete.Day.unique():
        dicty[str(i)]= {}

        for j in complete.Hour.unique():
            dicty[str(i)][str(j)] = {}
            for x in complete.StopID.unique():
                if x in ans[i][j]:
                    dicty[str(i)][str(j)][str(x)] = ans[i][j][x]
                else:
                    dicty[str(i)][str(j)][str(x)] = 0
    return dicty

In [3]:
df1 = pd.read_csv('/Users/jakek/Documents/Computer Science/Semester 3/Data Analytics/weather_csv_hourly.csv', low_memory=False)
df1['Date'], df1['Time'] = df1['DateTime'].str.split(' ', 1).str
df1.drop('DateTime', axis=1, inplace=True)
df1['Time'], x = df1['Time'].str.split(':', 1).str
df1['Time'] = df1.Time.astype(int)
df1 = df1.rename(columns={'Date': 'TimeFrame', 'Time': 'Hour'})


In [4]:
files = []
for file in  os.listdir("./complete dfs/"):
    if isfile("./complete dfs/"+file):
        file = file[:-4]
        files += [file]
files=set(files)

In [5]:
print(files)

{'44B0', '0081', '2380', '46A0', '0440', '1021', '32B0', '2361', '1450', '0180', '0611', '0391', '0610', '0750', '0160', '56A1', '0841', '51X1', '1510', '69X1', '0140', '0791', '1611', '32X1', '32B1', '45A0', '1511', '2390', '1451', '38A0', '1201', '1111', '1400', '0011', '1160', '0431', '1401', '0170', '41B0', '1841', '0400', '0090', '1020', '65B0', '1230', '66A1', '27X0', '83A1', '40B0', '0410', '0631', '0131', '0760', '1840', '0630', '0681', '84A0', '33B0', '79A0', '0470', '2391', '33A0', '1501', '67X0', '38A1', '77A0', '79A1', '1221', '1140', '0040', '33X1', '0390', '1041', '27X1', '77A1', '40D1', '41X0', '0840', '0130', '1161', '66B0', '68A1', '45A1', '0150', '2700', '0670', '0660', '0691', '2201', '66A0', '0680', '0690', '1850', '27B0', '51D0', '84X0', '84X1', '31B1', '1301', '32A1', '0590', '0761', '38B1', '32X0', '0071', '0430', '0260', '0661', '1231', '41X1', '66B1', '83A0', '1040', '0161', '0751', '0441', '0490', '2360', '1300', '39A0', '51D1', '1181', '27A1', '0110', '7470',

In [6]:
for file in files:
    print(file)
    if file+'cleaned.csv' in os.listdir("./complete dfs/cleaned/"):
        print("already exists")
        


44B0
already exists
0081
already exists
2380
already exists
46A0
already exists
0440
already exists
1021
already exists
32B0
already exists
2361
1450
already exists
0180
already exists
0611
already exists
0391
already exists
0610
0750
already exists
0160
already exists
56A1
0841
already exists
51X1
already exists
1510
69X1
already exists
0140
0791
1611
already exists
32X1
already exists
32B1
already exists
45A0
already exists
1511
already exists
2390
already exists
1451
already exists
38A0
already exists
1201
already exists
1111
already exists
1400
0011
already exists
1160
0431
1401
already exists
0170
already exists
41B0
already exists
1841
already exists
0400
already exists
0090
already exists
1020
65B0
already exists
1230
already exists
66A1
already exists
27X0
83A1
40B0
0410
0631
0131
already exists
0760
1840
already exists
0630
0681
84A0
33B0
already exists
79A0
0470
already exists
2391
already exists
33A0
already exists
1501
already exists
67X0
already exists
38A1
77A0
79A1
alrea

In [3]:
for file in files:
    print(file)
    if file+'cleaned.csv' in os.listdir("./complete dfs/cleaned/"):
        print("already exists")
    else:
        df = pd.read_csv('./complete dfs/'+file+'.csv', low_memory= False)
        df.drop('Unnamed: 0', axis=1, inplace=True)
        df, dict_real, unique_stops = compu(df)
        a = df.StopID.unique().tolist()
        b = set(unique_stops)
        ignore = list(set(a) ^ set(b))
        for i in ignore:
            df = df[df.StopID != i]
        df['TimeToNextStop'] = df.apply(lambda row: (dict_real[(row['VehicleJourneyID'], row['TimeFrame'], row['Instance'])][row['StopID']])*(10**-6), axis=1) 
        df= df.groupby(['VehicleJourneyID','TimeFrame', 'StopID']).apply(func)
        df['Hour'] = df.apply(lambda row: hour(row['Timestamp']), axis=1)
        df['Day'] = df.apply(lambda row: weekday(row['Timestamp']), axis=1)
        df = df.loc[df.TimeToNextStop != 0.0]
        ans = df.groupby(['Day','Hour','StopID']).TimeToNextStop.mean()
        df['Avg'] = df.apply(lambda row: average(row['Day'], row['Hour'], row['StopID']), axis=1)
        dicty = make_dict(df)
        dicty['index'] = stops(df)
        dicty['index'] = list(dicty['index'])
        for i in range(len(dicty['index'])):
            dicty['index'][i] = int(dicty['index'][i])
        with open('./JSON2/'+file+'.json', 'w') as fp:
            json.dump(dicty, fp)
        df = pd.merge(df,df1, how='left', on = ['TimeFrame','Hour'])
        df.to_csv("./complete dfs/cleaned/"+file+"cleaned.csv")

NameError: name 'files' is not defined

In [6]:
file = '0010'

print(file)
df = pd.read_csv('./complete dfs/'+file+'.csv', low_memory= False)
unique_stops = stops(df)
print(unique_stops)

0010


C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[ 226  228  229  227  230  231 1641 1642  213  214 4432  119   44   45   46
   47   48   49   50   51   52  265  271  340  350  351  352  353  354  355
  356  357  390  372  373  374  375 2804  376  377  378  380]


In [5]:
file = '0010'

print(file)
df = pd.read_csv('./complete dfs/'+file+'.csv', low_memory= False)
df, dict_real, unique_stops = compu(df)
print(unique_stops)

a = df.StopID.unique().tolist()
b = set(unique_stops)
ignore = list(set(a) ^ set(b))
for i in ignore:
    df = df[df.StopID != i]
df['TimeToNextStop'] = df.apply(lambda row: (dict_real[(row['VehicleJourneyID'], row['TimeFrame'], row['Instance'])][row['StopID']])*(10**-6), axis=1) 
df= df.groupby(['VehicleJourneyID','TimeFrame', 'StopID']).apply(func)
df['Hour'] = df.apply(lambda row: hour(row['Timestamp']), axis=1)
df['Day'] = df.apply(lambda row: weekday(row['Timestamp']), axis=1)
df = df.loc[df.TimeToNextStop != 0.0]
ans = df.groupby(['Day','Hour','StopID']).TimeToNextStop.mean()
df['Avg'] = df.apply(lambda row: average(row['Day'], row['Hour'], row['StopID']), axis=1)
dicty = make_dict(df)
dicty['index'] = stops(df)
print(dicty['index'])
dicty['index'] = list(dicty['index'])
print(dicty['index'])

0010


C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

## old code

In [139]:
# with open(json_path+ file+'.json') as data_file:    
#     dicty = json.load(data_file)

In [140]:
df.head()

,Unnamed: 0,Timestamp,LineID,Direction,JourneyPatternID,TimeFrame,VehicleJourneyID,Operator,Congestion,Lon,Lat,Delay,BlockID,VehicleID,StopID,AtStop,Instance
0,961,1.352188e+15,4.0,0,40001.0,2012-11-06,5712.0,CD,0.0,-6.277517,53.416718,0.0,4002.0,43033.0,324,1.0,1
1,966,1.352188e+15,4.0,0,40001.0,2012-11-06,5712.0,CD,0.0,-6.277517,53.416733,0.0,4002.0,43033.0,324,1.0,1
2,973,1.352188e+15,4.0,0,40001.0,2012-11-06,5712.0,CD,0.0,-6.277517,53.416733,0.0,4002.0,43033.0,324,1.0,1
3,976,1.352188e+15,4.0,0,40001.0,2012-11-06,5712.0,CD,0.0,-6.277517,53.416733,0.0,4002.0,43033.0,324,1.0,1
4,984,1.352188e+15,4.0,0,40001.0,2012-11-06,5712.0,CD,0.0,-6.277500,53.416718,0.0,4002.0,43033.0,324,1.0,1


In [141]:
def stops(file):
    maxi = 0
    for name, group in file.groupby(['VehicleJourneyID','TimeFrame','Instance']):
        if group['StopID'].nunique() == file['StopID'].nunique():
            return group['StopID'].unique()
        maxi = max(group['StopID'].nunique(), maxi)
        if group['StopID'].nunique() == maxi:
            group.sort_values(['Timestamp'], inplace = True)
            all_stops = group['StopID'].unique()
            if group['StopID'].nunique() == file['StopID'].nunique():
                break
    return all_stops

In [142]:
df.Instance.unique()

array([1, 2], dtype=int64)

In [143]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [144]:
# create a dictionary with the time to next stop
def calc(group):
    group.sort_values(['Timestamp'], inplace = True)  
    group = group[['VehicleJourneyID','TimeFrame','StopID','Timestamp','AtStop', 'Instance', 'Direction']]
    return group

def compu(df):
    total = []
    dict_real = {}
    unique_stops = stops(df)
    for name, group in df.groupby(['VehicleJourneyID','TimeFrame', 'Instance']):
        group = calc(group)
        dict_real[name] = {}
        for i in range(len(unique_stops)):
            if i != len(unique_stops)-1:
                start = group['Timestamp'].loc[group['StopID'] == unique_stops[i]].min()
                stop = group['Timestamp'].loc[group['StopID'] == unique_stops[i+1]].min()
                dict_real[name][unique_stops[i]] = stop-start
            else:
                dict_real[name][unique_stops[i]] = 0
        total += [group]
    complete = pd.concat(total)
    return complete

C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [145]:

a = complete.StopID.unique().tolist()
b = set(unique_stops)
ignore = list(set(a) ^ set(b))

5


In [146]:
for i in ignore:
    complete = complete[complete.StopID != i]

In [147]:
complete['TimeToNextStop'] = complete.apply(lambda row: (dict_real[(row['VehicleJourneyID'], row['TimeFrame'], row['Instance'])][row['StopID']])*(10**-6), axis=1)

In [148]:
print(unique_stops)

[ 324 7476 7478  327 7477 7479 7481 7482 7483 7113  127  112  113  114  115
   37   38   39   40  146  147  184  185  186  187  188  189  190  191  819
 7457 4725  273  402  405  408  409  410  411  412  413 7531 7532 7533 7536
  414  415  416  417  418  419  420  421  422  423  424  425  426  427  428
  429 6334 3114 3115 3116 6085 2026]


In [149]:
# only keep the first instance of each stop id for each group
def func(group):
    return group.loc[group['Timestamp'] == group['Timestamp'].min()]

complete= complete.groupby(['VehicleJourneyID','TimeFrame', 'StopID']).apply(func)

## create hour and time row

In [150]:
def hour(x):
    x = x/1000000
    date = datetime.datetime.fromtimestamp(x).strftime("%H")
    return(int(date))

complete['Hour'] = complete.apply(lambda row: hour(row['Timestamp']), axis=1)

In [151]:
def weekday(x):
    x = x/1000000
    x = int(x)
    if datetime.datetime.fromtimestamp(x).strftime("%A") == "Monday":
        return 1
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Tuesday":
        return 2
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Wednesday":
        return 3
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Thursday":
        return 4
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Friday":
        return 5
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Saturday":
        return 6
    elif datetime.datetime.fromtimestamp(x).strftime("%A") == "Sunday":
        return 7

complete['Day'] = complete.apply(lambda row: weekday(row['Timestamp']), axis=1)

In [152]:
complete.head()

VehicleJourneyID   TimeFrame  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101            4543.0  2013-01-02   
                            38     105108            4543.0  2013-01-02   
                            39     105114            4543.0  2013-01-02   
                            40     105130            4543.0  2013-01-02   
                            112    105055            4543.0  2013-01-02   

                                           StopID     Timestamp  AtStop  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101      37  1.357114e+15     1.0   
                            38     105108      38  1.357115e+15     0.0   
                            39     105114      39  1.357115e+15     0.0   
                            40     105130      40  1.357115e+15     1.0   
                            112    105055     112  1.357114e+15     1.0   

                                           Instance  Direction  \
VehicleJourneyID TimeFrame  StopID                               
4543.0           2013-01-02 37     105101         1          0   
                            38     105108         1          0   
                            39     105114         1          0   
                            40     105130         1          0   
                            112    105055         1          0   

                                           TimeToNextStop  Hour  Day  
VehicleJourneyID TimeFrame  StopID                                    
4543.0           2013-01-02 37     105101            45.0     8    3  
                            38     105108            30.0     8    3  
                            39     105114            44.0     8    3  
                            40     105130            62.0     8    3  
                            112    105055           100.0     8    3

In [153]:
complete = complete.loc[complete.TimeToNextStop != 0.0]

## now to create the better average feature

In [154]:
ans = complete.groupby(['Day','Hour','StopID']).TimeToNextStop.mean()

C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: 'StopID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  if __name__ == '__main__':


In [155]:
type(ans)

pandas.core.series.Series

In [156]:
def average(day, hour, stop):
    return ans[day][hour][stop]
complete['Avg'] = complete.apply(lambda row: average(row['Day'], row['Hour'], row['StopID']), axis=1)

In [157]:
complete.head()

VehicleJourneyID   TimeFrame  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101            4543.0  2013-01-02   
                            38     105108            4543.0  2013-01-02   
                            39     105114            4543.0  2013-01-02   
                            40     105130            4543.0  2013-01-02   
                            112    105055            4543.0  2013-01-02   

                                           StopID     Timestamp  AtStop  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101      37  1.357114e+15     1.0   
                            38     105108      38  1.357115e+15     0.0   
                            39     105114      39  1.357115e+15     0.0   
                            40     105130      40  1.357115e+15     1.0   
                            112    105055     112  1.357114e+15     1.0   

                                           Instance  Direction  \
VehicleJourneyID TimeFrame  StopID                               
4543.0           2013-01-02 37     105101         1          0   
                            38     105108         1          0   
                            39     105114         1          0   
                            40     105130         1          0   
                            112    105055         1          0   

                                           TimeToNextStop  Hour  Day  \
VehicleJourneyID TimeFrame  StopID                                     
4543.0           2013-01-02 37     105101            45.0     8    3   
                            38     105108            30.0     8    3   
                            39     105114            44.0     8    3   
                            40     105130            62.0     8    3   
                            112    105055           100.0     8    3   

                                                  Avg  
VehicleJourneyID TimeFrame  StopID                     
4543.0           2013-01-02 37     105101   73.333333  
                            38     105108   42.230769  
                            39     105114   95.482759  
                            40     105130   64.645161  
                            112    105055  137.233333

In [158]:
import winsound
Freq = 2500 
Dur = 1000 
winsound.Beep(Freq,Dur)


In [159]:
test = complete.loc[complete['Hour'] == 8]
test = test.loc[complete['Day'] == 3]
test = test.loc[complete['StopID'] == 37]
test.dropna(inplace=True)
test.head()

VehicleJourneyID   TimeFrame  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101            4543.0  2013-01-02   
                 2013-01-09 37     163177            4543.0  2013-01-09   
                 2013-01-16 37     223427            4543.0  2013-01-16   
                 2013-01-23 37     284651            4543.0  2013-01-23   
                 2013-01-30 37     339039            4543.0  2013-01-30   

                                           StopID     Timestamp  AtStop  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101      37  1.357114e+15     1.0   
                 2013-01-09 37     163177      37  1.357719e+15     1.0   
                 2013-01-16 37     223427      37  1.358324e+15     1.0   
                 2013-01-23 37     284651      37  1.358929e+15     0.0   
                 2013-01-30 37     339039      37  1.359534e+15     1.0   

                                           Instance  Direction  \
VehicleJourneyID TimeFrame  StopID                               
4543.0           2013-01-02 37     105101         1          0   
                 2013-01-09 37     163177         1          0   
                 2013-01-16 37     223427         1          0   
                 2013-01-23 37     284651         1          0   
                 2013-01-30 37     339039         1          0   

                                           TimeToNextStop  Hour  Day  \
VehicleJourneyID TimeFrame  StopID                                     
4543.0           2013-01-02 37     105101            45.0     8    3   
                 2013-01-09 37     163177            81.0     8    3   
                 2013-01-16 37     223427            82.0     8    3   
                 2013-01-23 37     284651            41.0     8    3   
                 2013-01-30 37     339039            80.0     8    3   

                                                 Avg  
VehicleJourneyID TimeFrame  StopID                    
4543.0           2013-01-02 37     105101  73.333333  
                 2013-01-09 37     163177  73.333333  
                 2013-01-16 37     223427  73.333333  
                 2013-01-23 37     284651  73.333333  
                 2013-01-30 37     339039  73.333333

In [160]:
test.shape

(27, 11)

In [161]:
test['TimeToNextStop'].sum()/test.shape[0]

73.33333333333333

In [162]:
def make_dict():
    dicty = {}
    for i in complete.Day.unique():
        dicty[str(i)]= {}

        for j in complete.Hour.unique():
            dicty[str(i)][str(j)] = {}
            for x in complete.StopID.unique():
                if x in ans[i][j]:
                    dicty[str(i)][str(j)][str(x)] = ans[i][j][x]
                else:
                    dicty[str(i)][str(j)][str(x)] = 0


In [163]:
print(dicty['2']['12'])

{'7536': 0, '127': 95.764705882352942, '421': 69.666666666666671, '38': 30.444444444444443, '115': 63.0, '186': 78.777777777777771, '427': 45.071428571428569, '39': 71.142857142857139, '428': 32.071428571428569, '7482': 68.0, '405': 140.875, '37': 72.705882352941174, '413': nan, '402': 116.0, '7478': 0, '191': 97.111111111111114, '419': 35.200000000000003, '425': 35.071428571428569, '409': 72.470588235294116, '416': 63.799999999999997, '429': 69.857142857142861, '3116': 77.933333333333337, '423': 59.571428571428569, '40': 32.692307692307693, '7532': 0, '420': 51.600000000000001, '7533': 0, '410': 37.25, '6085': 70.666666666666671, '327': -119.0, '3114': 35.06666666666667, '7531': nan, '414': 38.384615384615387, '185': 61.666666666666664, '7481': 98.0, '418': 48.799999999999997, '113': 34.071428571428569, '3115': 68.3125, '7113': 79.058823529411768, '7483': 24.600000000000001, '411': 98.071428571428569, '190': 87.0, '147': 91.944444444444443, '7479': 60.0, '7477': 41.0, '189': 27.764705

In [164]:
def stops(file):
    maxi = 0
    for name, group in file.groupby(['VehicleJourneyID','TimeFrame']):
        if group['StopID'].nunique() == file['StopID'].nunique():
            return group['StopID'].unique()
        maxi = max(group['StopID'].nunique(), maxi)
        if group['StopID'].nunique() == maxi:
            group.sort_values(['Timestamp'], inplace = True)
            all_stops = group['StopID'].unique()
            if group['StopID'].nunique() == file['StopID'].nunique():
                break
    return all_stops

In [165]:
dicty['index'] = stops(df)

dicty['index'] = list(dicty['index'])

for i in range(len(dicty['index'])):
    dicty['index'][i] = int(dicty['index'][i])

with open('./JSON2/bus1/'+file+'.json', 'w') as fp:
    json.dump(dicty, fp)

C:\Users\jakek\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [166]:
print(dicty)

{'7': {'7': {'7536': 0, '127': 0, '421': 0, '38': 0, '115': 0, '186': 0, '427': 0, '39': 0, '428': 0, '7482': 0, '405': 0, '37': 0, '413': 0, '402': 0, '7478': 0, '191': 0, '419': 0, '425': 0, '409': 0, '416': 0, '429': 0, '3116': 0, '423': 0, '40': 0, '7532': 0, '420': 0, '7533': 0, '410': 0, '6085': 0, '327': 0, '3114': 0, '7531': 0, '414': 0, '185': 0, '7481': 0, '418': 0, '113': 0, '3115': 0, '7113': 0, '7483': 0, '411': 0, '190': 0, '147': 0, '7479': 0, '7477': 0, '189': 0, '426': 0, '324': 0, '273': 0, '417': 0, '187': 0, '4725': 0, '424': 0, '7457': 0, '422': 0, '184': 0, '188': 0, '408': 0, '819': 0, '7476': 0, '114': 0, '146': 0, '415': 0, '412': 0, '112': 0, '6334': 0}, '11': {'7536': 0, '127': 84.0, '421': 48.142857142857146, '38': 26.333333333333332, '115': 52.333333333333336, '186': 30.166666666666668, '427': 19.75, '39': 109.5, '428': 27.625, '7482': 102.0, '405': 76.200000000000003, '37': 50.666666666666664, '413': nan, '402': 94.857142857142861, '7478': 83.5, '191': 66.

In [167]:
complete.head()

VehicleJourneyID   TimeFrame  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101            4543.0  2013-01-02   
                            38     105108            4543.0  2013-01-02   
                            39     105114            4543.0  2013-01-02   
                            40     105130            4543.0  2013-01-02   
                            112    105055            4543.0  2013-01-02   

                                           StopID     Timestamp  AtStop  \
VehicleJourneyID TimeFrame  StopID                                        
4543.0           2013-01-02 37     105101      37  1.357114e+15     1.0   
                            38     105108      38  1.357115e+15     0.0   
                            39     105114      39  1.357115e+15     0.0   
                            40     105130      40  1.357115e+15     1.0   
                            112    105055     112  1.357114e+15     1.0   

                                           Instance  Direction  \
VehicleJourneyID TimeFrame  StopID                               
4543.0           2013-01-02 37     105101         1          0   
                            38     105108         1          0   
                            39     105114         1          0   
                            40     105130         1          0   
                            112    105055         1          0   

                                           TimeToNextStop  Hour  Day  \
VehicleJourneyID TimeFrame  StopID                                     
4543.0           2013-01-02 37     105101            45.0     8    3   
                            38     105108            30.0     8    3   
                            39     105114            44.0     8    3   
                            40     105130            62.0     8    3   
                            112    105055           100.0     8    3   

                                                  Avg  
VehicleJourneyID TimeFrame  StopID                     
4543.0           2013-01-02 37     105101   73.333333  
                            38     105108   42.230769  
                            39     105114   95.482759  
                            40     105130   64.645161  
                            112    105055  137.233333

In [168]:
df1 = pd.read_csv('/Users/jakek/Documents/Computer Science/Semester 3/Data Analytics/weather_csv_hourly.csv', low_memory=False)
df1['Date'], df1['Time'] = df1['DateTime'].str.split(' ', 1).str
df1.drop('DateTime', axis=1, inplace=True)
df1['Time'], x = df1['Time'].str.split(':', 1).str
df1['Time'] = df1.Time.astype(int)

In [169]:
df1 = df1.rename(columns={'Date': 'TimeFrame', 'Time': 'Hour'})
complete = pd.merge(complete,df1, how='left', on = ['TimeFrame','Hour'])

In [170]:
complete.head()

,VehicleJourneyID,TimeFrame,StopID,Timestamp,AtStop,Instance,Direction,TimeToNextStop,Hour,Day,Avg,Summary,Temp,Rain,Wind_Speed
0,4543.0,2013-01-02,37,1.357114e+15,1.0,1,0,45.0,8,3,73.333333,Light Drizzle,9.0,0.0,33.0
1,4543.0,2013-01-02,38,1.357115e+15,0.0,1,0,30.0,8,3,42.230769,Light Drizzle,9.0,0.0,33.0
2,4543.0,2013-01-02,39,1.357115e+15,0.0,1,0,44.0,8,3,95.482759,Light Drizzle,9.0,0.0,33.0
3,4543.0,2013-01-02,40,1.357115e+15,1.0,1,0,62.0,8,3,64.645161,Light Drizzle,9.0,0.0,33.0
4,4543.0,2013-01-02,112,1.357114e+15,1.0,1,0,100.0,8,3,137.233333,Light Drizzle,9.0,0.0,33.0


In [171]:
complete.to_csv("./complete dfs/bus 1/"+file+"cleaned.csv")